In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# -*- coding: utf-8 -*-
"""
@author: DIEGO BORJA

"""
#Carga de librerías
import numpy as np
import pandas as pd

#carga de datos
proyectos = pd.DataFrame(pd.read_csv('/content/drive/My Drive/SENA/SENNOVA/EVIDENCIAS/2019_2/eval_proyec/proyectos.csv'))
evaluadores = pd.DataFrame(pd.read_csv('/content/drive/My Drive/SENA/SENNOVA/EVIDENCIAS/2019_2/eval_proyec/evaluadores.csv'))
#Emparejamiento por Red de Conocimiento
merge_RC = pd.merge(proyectos[['ID_PROYECTO','ID_RC','ID_REG']], evaluadores[['ID_EVALUADOR','ID_RC','ID_REG']], on='ID_RC')
#Emparejamiento por Área de aplicación
merge_AP = pd.merge(proyectos[['ID_PROYECTO','ID_AP','ID_REG']], evaluadores[['ID_EVALUADOR','ID_AP','ID_REG']], on='ID_AP')
#Unión de tablas
df_merge = pd.concat([merge_RC,merge_AP], ignore_index=True,sort=False)
#verificar Regional
df_merge['Repite_Regional'] = np.where(df_merge['ID_REG_x']==df_merge['ID_REG_y'], 'SI', 'NO')
#Ordenar por 'ID_PROYECTO'
df_merge = df_merge.sort_values(by=['ID_PROYECTO'])
#Reordenar Índice
df_merge = df_merge.reset_index()
#Unión de tablas
df_merge = df_merge[['ID_PROYECTO','ID_RC','ID_AP','ID_EVALUADOR','ID_REG_y','ID_REG_x','Repite_Regional']]
#Reemplazar NaN por Cero (0)
df_merge = df_merge.fillna(0)
print('\nEmparejamiento por Red de Conocimiento y Área de aplicación'.upper())
df_merge.head(30)


EMPAREJAMIENTO POR RED DE CONOCIMIENTO Y ÁREA DE APLICACIÓN


,ID_PROYECTO,ID_RC,ID_AP,ID_EVALUADOR,ID_REG_y,ID_REG_x,Repite_Regional
0,1,3.0,0.0,6,14,25,NO
1,1,3.0,0.0,22,20,25,NO
2,1,3.0,0.0,50,20,25,NO
3,1,0.0,5.0,1,16,25,NO
4,1,0.0,5.0,10,14,25,NO
5,1,0.0,5.0,19,10,25,NO
6,1,0.0,5.0,22,20,25,NO
7,1,0.0,5.0,38,19,25,NO
8,1,0.0,5.0,53,4,25,NO
9,2,0.0,4.0,13,3,28,NO


In [3]:
lista = [1,6]
evaluadores[evaluadores['ID_EVALUADOR'].isin(lista)]

,ID_EVALUADOR,ID_RC,ID_AP,ID_REG
0,1,10,5,16
5,6,3,20,14


In [4]:
proyectos[proyectos['ID_PROYECTO'] == 1]

,ID_PROYECTO,ID_RC,ID_AP,ID_REG
0,1,3,5,25


In [5]:
#Emparejamiento
print('\nTodas las posibles combinaciones de emparejamiento'.upper())
#Crear tabla en blanco
eval_proy = pd.DataFrame(columns=['ID_PROYECTO', 'EVALUADOR_RC', 'EVALUADOR_AP'])
#Número de proyectos
n = proyectos.shape[0]

for k in range(1,n+1):
    #Seleccionar datos por Proyecto
    df = df_merge[df_merge['ID_PROYECTO'] == k]
    #Tamaño de la tabla
    t = df.shape[0]
    for i in range(t):
        for j in range(t):
            if i!=j:
                #Verificando diferentes evaluadores
                if (df.iloc[i]['ID_EVALUADOR']!=df.iloc[j]['ID_EVALUADOR']):
                    #Verificando Evaluador por Red de Conocimiento y Regional
                    if (df.iloc[i]['ID_RC'] != 0) and (df.iloc[i]['Repite_Regional'] == 'NO'):
                        #Verificando Evaluador por Área de aplicación y Regional
                        if (df.iloc[j]['ID_AP'] != 0) and (df.iloc[j]['Repite_Regional'] =='NO'):
                            #Todos los posibles emparejamientos por Proyecto
                            eval_proy = eval_proy.append({'ID_PROYECTO':df.iloc[i]['ID_PROYECTO'],'EVALUADOR_RC':df.iloc[i]['ID_EVALUADOR'],'EVALUADOR_AP':df.iloc[j]['ID_EVALUADOR']}, ignore_index=True)
#Convertir a tipo numérico
eval_proy = eval_proy.astype(int)


TODAS LAS POSIBLES COMBINACIONES DE EMPAREJAMIENTO


In [6]:
eval_proy.head(30)

,ID_PROYECTO,EVALUADOR_RC,EVALUADOR_AP
0,1,6,1
1,1,6,10
2,1,6,19
3,1,6,22
4,1,6,38
5,1,6,53
6,1,22,1
7,1,22,10
8,1,22,19
9,1,22,38


In [7]:
#Remuestreo aleatorio de proyectos
proy = eval_proy.sample(frac=1).reset_index(drop=True)
resultados=proy.drop_duplicates(['ID_PROYECTO'], keep='first')
resultados.sort_values(by=['ID_PROYECTO']).head()

,ID_PROYECTO,EVALUADOR_RC,EVALUADOR_AP
1254,1,22,10
2063,2,32,37
7305,6,58,30
1104,7,19,30
1788,8,22,11


In [8]:
#Conteo de Proyectos por Evaluador
print('\nConteo de proyectos asignados por evaluador'.upper())
dif = 90
#Crear tabla en blanco
num_proy = pd.DataFrame()

while dif > 85:
    #Remuestreo aleatorio de proyectos
    proy = eval_proy.sample(frac=1).reset_index(drop=True)
    #Eliminación de proyectos repetidos
    resultados=proy.drop_duplicates(['ID_PROYECTO'], keep='last')
    #Conteo proyectos por Evaluador por Red de Conocimiento
    cont_RC = pd.DataFrame(resultados['EVALUADOR_RC'].value_counts()).sort_index(axis=0)
    #Conteo proyectos por Evaluador por Red de Conocimiento
    cont_AP = pd.DataFrame(resultados['EVALUADOR_AP'].value_counts()).sort_index(axis=0) 
    #Total de proyectos por Evaluador
    num_proy['TOTAL_PROYECTOS'] = cont_RC['EVALUADOR_RC'] + cont_AP['EVALUADOR_AP']
    #Resumen de conteo
    conteo = num_proy.describe()
    #Número máximo de proyectos asignados para un evaluador
    min = conteo.iloc[3]
    #Número máximo de proyectos asignados para un evaluador
    max = conteo.iloc[7]
    #Máxima diferencia del número de proyectos asignados
    dif = float(max - min)
    #print(dif)


CONTEO DE PROYECTOS ASIGNADOS POR EVALUADOR


In [9]:
cont_RC.head()

,EVALUADOR_RC
1,17
2,26
3,21
4,21
5,12


In [10]:
cont_AP.head()

,EVALUADOR_AP
1,20
2,16
3,32
4,18
5,10


In [11]:
num_proy['ID_EVALUADOR'] = num_proy.index
num_proy = num_proy[['ID_EVALUADOR','TOTAL_PROYECTOS']]
num_proy.head(60)

,ID_EVALUADOR,TOTAL_PROYECTOS
1,1,37
2,2,42
3,3,53
4,4,39
5,5,22
6,6,22
7,7,22
8,8,28
9,9,31
10,10,48


In [12]:
conteo = num_proy['TOTAL_PROYECTOS'].describe()
conteo

count     60.000000
mean      36.333333
std       16.586360
min       19.000000
25%       24.750000
50%       31.000000
75%       41.250000
max      104.000000
Name: TOTAL_PROYECTOS, dtype: float64

In [13]:
min = conteo.iloc[3]
    #Número máximo de proyectos asignados para un evaluador
max = conteo.iloc[7]
    #Máxima diferencia del número de proyectos asignados
dif = float(max - min)
print(dif)

85.0


In [0]:
#Proyectos sin emparejamiento
pendiente = pd.concat([proyectos['ID_PROYECTO'], resultados['ID_PROYECTO']],ignore_index=True).drop_duplicates(keep=False)
pendiente = pd.DataFrame(pendiente)
#Proyectos sin emparejamiento pero con al menos un evaluador asociado
parcial = pendiente.merge(df_merge)
print('\033[1;32m PROYECTOS PARCIALES')
parcial.head()

In [15]:
huerfanos = pd.concat([pendiente['ID_PROYECTO'], parcial['ID_PROYECTO']],ignore_index=True).drop_duplicates(keep=False)
huerfanos = pd.DataFrame(huerfanos)
print('\033[1;91m PROYECTOS HUERFANOS...')
huerfanos

 PROYECTOS HUERFANOS...


,ID_PROYECTO
0,3
17,52
20,72
31,121
34,132
35,141
110,433
149,626
152,634
170,722


In [0]:
no_match = pd.DataFrame()
no_match['ID_PROYECTO'] = resultados['ID_PROYECTO']
no_list = no_match['ID_PROYECTO'].tolist()
no_match = proyectos[~proyectos['ID_PROYECTO'].isin(no_list)]
no_match.head()

,ID_PROYECTO,ID_RC,ID_AP,ID_REG
2,3,30,7,18
3,4,16,14,10
4,5,17,7,13
8,9,16,13,15
14,15,27,7,29
